# viz-1b-flights-splom.ipynb

This notebook builds on the learnings from [the first one](ttps://walterra.github.io/jupyter2kibana/viz-1a-flights-histogram.html) and uses the same data to create and deploy a scatterplot matrix. The difference here is that we moved the code to create the SavedObject in Kibana to a helper function into [kibana_vega_util.py](https://github.com/walterra/jupyter2kibana/blob/master/kibana_vega_util.py) so we can reuse it.

In [11]:
import datetime
import altair as alt
import eland as ed
import json
import numpy as np
import matplotlib.pyplot as plt
import urllib3

alt.data_transformers.disable_max_rows()
# Only in development environments with self signed certificates enable this setting to disable warnings
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

DataTransformerRegistry.enable('default')

The following cell will only output a visualization populated with data if the ES instance referenced has security disabled (not recommended, do only for demo purposes). The cell still needs to be run to create the Vega specification used in the cell after that to save the visualization in Kibana.

In [24]:
# Note: To create the Vega spec using Altair we reference ES via URL first. This will only work
# for non-secured ES instances. If your ES instance runs using SSL and/or authentication the chart
# in this cell will render empty. You can still save the visualization in Kibana correctly in the
# next cell because there the URL gets replaced with an Elasticsearch query
# to be used via the Kibana Vega plugin.

# WARNING:
# Do the following approach using a proxy only for demo purposes in a development environment.
# It will expose a secured ES instance unsecured!
# To make this work for demo purposes run the nodejs based proxy in a separate terminal like this:
# NODE_TLS_REJECT_UNAUTHORIZED='0' node proxy

url = 'http://localhost:9220/kibana_sample_data_flights/_search?size=1000'
url_data = alt.Data(url=url, format=alt.DataFormat(property='hits.hits',type='json'))

fields = [
    'Carrier',
    'AvgTicketPrice',
    'DistanceKilometers',
    'DistanceMiles',
    'FlightDelayMin',
    'FlightTimeMin',
    'dayOfWeek'
]

rename_dict = dict((a, 'datum._source.'+a) for a in fields)

chart = alt.Chart(url_data).mark_circle(
    opacity=.5,
    size=6
).transform_calculate(**rename_dict).encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative')
).properties(
    width=100,
    height=100
).repeat(
    row=['AvgTicketPrice', 'DistanceKilometers', 'FlightDelayMin', 'FlightTimeMin'],
    column=['FlightTimeMin', 'FlightDelayMin', 'DistanceKilometers', 'AvgTicketPrice']
).interactive()

chart

alt.RepeatChart(...)

In [25]:
from kibana_vega_util import saveVegaVis

saveVegaVis(
    'kibana_sample_data_flights',
    'def-vega-splom-1',
    chart,
    resultSize=1000,
    # Only in development environments with self signed certificates fall back to use `verify=False`
    # verify=False
)

<Response [409]>